## Tic-Tac-Toe Agent
​
In this notebook, you will learn to build an RL agent (using Q-learning) that learns to play Numerical Tic-Tac-Toe with odd numbers. The environment is playing randomly with the agent, i.e. its strategy is to put an even number randomly in an empty cell. The following is the layout of the notebook:
        - Defining epsilon-greedy strategy
        - Tracking state-action pairs for convergence
        - Define hyperparameters for the Q-learning algorithm
        - Generating episode and applying Q-update equation
        - Checking convergence in Q-values

#### Importing libraries
Write the code to import Tic-Tac-Toe class from the environment file

In [206]:
from TCGame_Env1 import TicTacToe  # from <TC_Env> import <TicTacToe> - import your class from environment file
import collections
import numpy as np
import random
import pickle
import time
from matplotlib import pyplot as plt

## defining object for the Tictac toe environment 
env = TicTacToe()

In [207]:
# Function to convert state array into a string to store it as keys in the dictionary
# states in Q-dictionary will be of form: x-4-5-3-8-x-x-x-x
#   x | 4 | 5
#   ----------
#   3 | 8 | x
#   ----------
#   x | x | x

def Q_state(state):

    return ('-'.join(str(e) for e in state)).replace('nan','x')

In [208]:
# Defining a function which will return valid (all possible actions) actions corresponding to a state
# Important to avoid errors during deployment.

def valid_actions(state):

    valid_Actions = []
    
    valid_Actions = [i for i in env.action_space(state)[0]] ###### -------please call your environment as env
    return valid_Actions

In [209]:
# Defining a function which will add new Q-values to the Q-dictionary. 
def add_to_dict(state):
    state1 = Q_state(state)
    
    valid_act = valid_actions(state)
    if state1 not in Q_dict.keys():
        for action in valid_act:
            Q_dict[state1][action]=0

In [210]:
# printing the default tic tac toe board positions
Q_state(env.state)

'x-x-x-x-x-x-x-x-x'

In [211]:
# Printing all the valid actions
valid_actions(env.state)

[(0, 1),
 (0, 3),
 (0, 5),
 (0, 7),
 (0, 9),
 (1, 1),
 (1, 3),
 (1, 5),
 (1, 7),
 (1, 9),
 (2, 1),
 (2, 3),
 (2, 5),
 (2, 7),
 (2, 9),
 (3, 1),
 (3, 3),
 (3, 5),
 (3, 7),
 (3, 9),
 (4, 1),
 (4, 3),
 (4, 5),
 (4, 7),
 (4, 9),
 (5, 1),
 (5, 3),
 (5, 5),
 (5, 7),
 (5, 9),
 (6, 1),
 (6, 3),
 (6, 5),
 (6, 7),
 (6, 9),
 (7, 1),
 (7, 3),
 (7, 5),
 (7, 7),
 (7, 9),
 (8, 1),
 (8, 3),
 (8, 5),
 (8, 7),
 (8, 9)]

#### Epsilon-greedy strategy - Write your code here

(you can build your epsilon-decay function similar to the one given at the end of the notebook)

In [212]:
# Defining epsilon-greedy policy. You can choose any function epsilon-decay strategy
def epsilon_greedy(state, time):
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate*time)
    # print(epsilon)
    z = np.random.random()
        
    if z > epsilon:
        state1 = Q_state(state)
        action = max(Q_dict[state1],key=Q_dict[state1].get)   #Exploitation: this gets the action corresponding to max q-value of current state
    else:
        possible_action = [i for i in env.action_space(state)[0]]        
        action = possible_action[np.random.choice(range(len(possible_action)))]    #Exploration: randomly choosing and action
    
    return action

#### Tracking the state-action pairs for checking convergence - write your code here

In [213]:
# Initialise Q_dictionary as 'Q_dict' and States_tracked as 'States_track' (for convergence)
Q_dict = collections.defaultdict(dict)
States_track =collections.defaultdict(dict)

In [214]:
# Initialise states to be tracked
def initialise_tracking_states():
    Sample_Qvalues = [('x-x-x-x-x-x-x-x-x',(5,7)),('x-x-x-x-x-x-x-x-x',(7,9)),
                       ('x-3-x-x-1-x-x-x-x',(2,3)),('x-5-x-x-x-x-5-7-x',(8,1))]    #selecting random 4 Q-values
    for q_values in Sample_Qvalues:
        state = q_values[0]
        action = q_values[1]
        States_track[state][action] = []

In [215]:
#Defining a function to save the Q-dictionary as a pickle file

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [216]:
def save_tracking_states():
    for state in States_track.keys():
        for action in States_track[state].keys():
            if state in Q_dict and action in Q_dict[state]:
                States_track[state][action].append(Q_dict[state][action])

In [217]:
initialise_tracking_states()

#### Define hyperparameters  ---write your code here

In [218]:
#Defining parameters for the training

EPISODES = 5000000
LR = 0.01                   # learning rate
GAMMA = 0.8                # discount factor

max_epsilon = 1.0
min_epsilon = 0.001
decay_rate = 0.001        # epsilon decay rate
threshold = 200          # no of episodes after which states_tracked wwill be saved
policy_threshold = 300   # no of episodes after which Q dictionary/table will be saved

### Q-update loop ---write your code here

In [ ]:
start_time = time.time()

for episode in range(EPISODES):
    ##### Start writing your code from the next line
    
    env = TicTacToe() # call the environment
    curr_state = env.state    
    done = False  
    add_to_dict(curr_state)  # adding the current state to dictionary

    while done != True:  
        curr_state1 = Q_state(curr_state)
        curr_action = epsilon_greedy(curr_state, episode)   # applying epislon method
        next_state, reward, done = env.step(curr_state, curr_action) # getting reward
        next_state_temp = Q_state(next_state)
        add_to_dict(next_state)

        # Updating rules
        if done != True:
            max_next = max(Q_dict[next_state_temp],key=Q_dict[next_state_temp].get)  
                #this gets the action corresponding to max q-value of next state
            Q_dict[curr_state1][curr_action] += LR * ((reward + (GAMMA*(Q_dict[next_state_temp][max_next])))
                                                      - Q_dict[curr_state1][curr_action] ) 
        else:
            Q_dict[curr_state1][curr_action] += LR * ((reward - Q_dict[curr_state1][curr_action]))

        # navigating to next state
        curr_state = next_state
        
     #states tracking   
    if ((episode+1)%threshold)==0:
        save_tracking_states()
        save_obj(States_track,'States_tracking')
        print(episode) 

    if ((episode+1)% policy_threshold) == 0: 
        save_obj(Q_dict,'Policy_Q_dict')     
    
elapsed_time = time.time() - start_time
save_obj(States_track,'States_tracked')   
save_obj(Q_dict,'Policy')

199
399
599
799
999
1199
1399
1599
1799
1999
2199
2399
2599
2799
2999
3199
3399
3599
3799
3999
4199
4399
4599
4799
4999
5199
5399
5599
5799
5999
6199
6399
6599
6799
6999
7199
7399
7599
7799
7999
8199
8399
8599
8799
8999
9199
9399
9599
9799
9999
10199
10399
10599
10799
10999
11199
11399
11599
11799
11999
12199
12399
12599
12799
12999
13199
13399
13599
13799
13999
14199
14399
14599
14799
14999
15199
15399
15599
15799
15999
16199
16399
16599
16799
16999
17199
17399
17599
17799
17999
18199
18399
18599
18799
18999
19199
19399
19599
19799
19999
20199
20399
20599
20799
20999
21199
21399
21599
21799
21999
22199
22399
22599
22799
22999
23199
23399
23599
23799
23999
24199
24399
24599
24799
24999
25199
25399
25599
25799
25999
26199
26399
26599
26799
26999
27199
27399
27599
27799
27999
28199
28399
28599
28799
28999
29199
29399
29599
29799
29999
30199
30399
30599
30799
30999
31199
31399
31599
31799
31999
32199
32399
32599
32799
32999
33199
33399
33599
33799
33999
34199
34399
34599
34799
34999
35199

250399
250599
250799
250999
251199
251399
251599
251799
251999
252199
252399
252599
252799
252999
253199
253399
253599
253799
253999
254199
254399
254599
254799
254999
255199
255399
255599
255799
255999
256199
256399
256599
256799
256999
257199
257399
257599
257799
257999
258199
258399
258599
258799
258999
259199
259399
259599
259799
259999
260199
260399
260599
260799
260999
261199
261399
261599
261799
261999
262199
262399
262599
262799
262999
263199
263399
263599
263799
263999
264199
264399
264599
264799
264999
265199
265399
265599
265799
265999
266199
266399
266599
266799
266999
267199
267399
267599
267799
267999
268199
268399
268599
268799
268999
269199
269399
269599
269799
269999
270199
270399
270599
270799
270999
271199
271399
271599
271799
271999
272199
272399
272599
272799
272999
273199
273399
273599
273799
273999
274199
274399
274599
274799
274999
275199
275399
275599
275799
275999
276199
276399
276599
276799
276999
277199
277399
277599
277799
277999
278199
278399
278599
278799

484599
484799
484999
485199
485399
485599
485799
485999
486199
486399
486599
486799
486999
487199
487399
487599
487799
487999
488199
488399
488599
488799
488999
489199
489399
489599
489799
489999
490199
490399
490599
490799
490999
491199
491399
491599
491799
491999
492199
492399
492599
492799
492999
493199
493399
493599
493799
493999
494199
494399
494599
494799
494999
495199
495399
495599
495799
495999
496199
496399
496599
496799
496999
497199
497399
497599
497799
497999
498199
498399
498599
498799
498999
499199
499399
499599
499799
499999
500199
500399
500599
500799
500999
501199
501399
501599
501799
501999
502199
502399
502599
502799
502999
503199
503399
503599
503799
503999
504199
504399
504599
504799
504999
505199
505399
505599
505799
505999
506199
506399
506599
506799
506999
507199
507399
507599
507799
507999
508199
508399
508599
508799
508999
509199
509399
509599
509799
509999
510199
510399
510599
510799
510999
511199
511399
511599
511799
511999
512199
512399
512599
512799
512999

718999
719199
719399
719599
719799
719999
720199
720399
720599
720799
720999
721199
721399
721599
721799
721999
722199
722399
722599
722799
722999
723199
723399
723599
723799
723999
724199
724399
724599
724799
724999
725199
725399
725599
725799
725999
726199
726399
726599
726799
726999
727199
727399
727599
727799
727999
728199
728399
728599
728799
728999
729199
729399
729599
729799
729999
730199
730399
730599
730799
730999
731199
731399
731599
731799
731999
732199
732399
732599
732799
732999
733199
733399
733599
733799
733999
734199
734399
734599
734799
734999
735199
735399
735599
735799
735999
736199
736399
736599
736799
736999
737199
737399
737599
737799
737999
738199
738399
738599
738799
738999
739199
739399
739599
739799
739999
740199
740399
740599
740799
740999
741199
741399
741599
741799
741999
742199
742399
742599
742799
742999
743199
743399
743599
743799
743999
744199
744399
744599
744799
744999
745199
745399
745599
745799
745999
746199
746399
746599
746799
746999
747199
747399

#### Check the Q-dictionary

In [ ]:
Q_dict

In [ ]:
len(Q_dict)

In [ ]:
# try checking for one of the states - that which action your agent thinks is the best  -----This will not be evaluated

#### Check the states tracked for Q-values convergence
(non-evaluative)

In [ ]:
# Write the code for plotting the graphs for state-action pairs tracked

In [ ]:
plt.figure(0, figsize=(16,7))

x_axis = np.asarray(range(0, len(States_track['x-x-x-x-x-x-x-x-x'][(5,7)])))
plt.subplot(221)
plt.plot(x_axis,np.asarray(States_track['x-x-x-x-x-x-x-x-x'][(5,7)]))
plt.show

x_axis = np.asarray(range(0, len(States_track['x-3-x-x-1-x-x-x-x'][(2,3)])))
plt.subplot(222)
plt.plot(x_axis,np.asarray(States_track['x-3-x-x-1-x-x-x-x'][(2,3)]))
plt.show

x_axis = np.asarray(range(0, len(States_track['x-x-x-x-x-x-x-x-x'][(7,9)])))
plt.subplot(223)
plt.plot(x_axis,np.asarray(States_track['x-x-x-x-x-x-x-x-x'][(7,9)]))
plt.show

x_axis = np.asarray(range(0, len(States_track['x-5-x-x-x-x-5-7-x'][(8,1)])))
plt.subplot(224)
plt.plot(x_axis,np.asarray(States_track['x-5-x-x-x-x-5-7-x'][(8,1)]))
plt.show

### Epsilon - decay check

In [ ]:
max_epsilon = 1.0
min_epsilon = 0.001
time = np.arange(0,5000000)
epsilon = []
for i in range(0,5000000):
    epsilon.append(min_epsilon + (max_epsilon - min_epsilon) * np.exp(-0.000001*i))

In [ ]:
plt.plot(time, epsilon)
plt.show()